# SeeMe.ai quick guide for fast.ai with dataset download.

# 1. Setup

## 1.1 Where to run your Jupyter Notebooks

Use your preferred platfrom to run your [Jupyter Notebooks](https://jupyter.org/): 

* [Paperspace](https://paperspace.com)
* [Google Colab](https://colab.research.google.com/)
* [The official Fast.ai Docker images](https://hub.docker.com/u/fastdotai)
* [Fast.ai Docker from SeeMe.ai](https://hub.docker.com/repository/docker/seemeai/fastai)
* [Azure](https://azure.microsoft.com)
* ...

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 1.9.0


In [3]:
!pip install --upgrade fastai

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [4]:
import fastai
print(f"fastai version: {fastai.__version__}")

fastai version: 2.5.2


## 1.2 SeeMe.ai Python SDK

Install the [SeeMe SDK](https://pypi.org/project/seeme/) from the command line:

```bash
$ pip install --upgrade --no-cache-dir seeme
```

or in your Notebook:

In [5]:
# Install the SeeMe SDK from your notebook.
!pip install --upgrade --no-cache-dir seeme

  Attempting uninstall: seeme
    Found existing installation: seeme 0.12.9
    Uninstalling seeme-0.12.9:
      Successfully uninstalled seeme-0.12.9
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


# 2. Create a client

In [6]:
from seeme import Client

In [7]:
cl = Client(username="janvdp", apikey="a8b4f486-87a8-47dd-beaa-596086cfc146")

In [8]:
#cl.login("janvdp", "")

In [9]:
dataset_id = "f3d3043a-9344-4365-8147-8b6f3818a95f"
version_id = "b531f745-aead-48d7-a34f-f65d73518255"
data_folder = "un_flags"

In [10]:
cl.download_dataset(dataset_id, version_id, extract_to_dir=data_folder)

In [11]:
# Import the vision modules
from fastai.vision.all import *

In [12]:
path = Path(data_folder)

In [13]:
path.ls()

(#2) [Path('un_flags/train'),Path('un_flags/valid')]

## 2.1 Train our Flags model

Here we will train a basic flag model, using the defaults from fastai.

In [14]:
# Create the dataloaders
# for more details check the fast.ai courses
dls = ImageDataLoaders.from_folder(path, seed=12, item_tfms=Resize(460),
                                    batch_tfms=aug_transforms(size=224))

/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:1023: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:760.)
  ret = func(*args, **kwargs)


In [15]:
# Create your model
learn =cnn_learner(
    dls,
    resnet50,
    metrics=accuracy,
    cbs=[SaveModelCallback(monitor="accuracy", fname="best_flags")]
)

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [16]:
# Fine tune your model
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,accuracy,time
0,2.385049,0.926311,0.788321,01:10
1,1.128045,0.580653,0.874348,01:09


Better model found at epoch 0 with accuracy value: 0.7883211970329285.
Better model found at epoch 1 with accuracy value: 0.8743482828140259.


# 3. Deployment - SeeMe.ai

Once you have trained your model, it's time to get it into production.

## 3.1 Export your model

In [ ]:
# Put your model in eval model
learn.model.eval();

In [ ]:
# Export your model
learn.export()

In [20]:
flag_model = cl.get_model('94f0fb9b-c93d-4078-863a-415ddc154cb6')

In [19]:
# models = cl.get_models()
# flag_model = [ model for model in models if "Flags" in model["name"]]
# flag_model

[{'id': '94f0fb9b-c93d-4078-863a-415ddc154cb6',
  'created_at': '2020-05-01T11:40:51.49792Z',
  'update_at': '2021-10-14T11:39:08.111272Z',
  'deleted_at': None,
  'name': 'Flags of the world',
  'description': 'UN countries flag classification. Detect all the flags in the world and get more details.',
  'active_version_id': '0b7c2371-09b1-481d-9598-62d0acb8c2e1',
  'classes': 'Flags of the world',
  'count': 0,
  'precision': 0,
  'recall': 0,
  'accuracy': 0,
  'user_id': 'd7159432-f218-44ac-aebe-e5d661d62862',
  'can_inference': True,
  'kind': 'image_classification',
  'has_logo': True,
  'logo': '94f0fb9b-c93d-4078-863a-415ddc154cb6.jpg',
  'public': True,
  'config': '',
  'application_id': 'feb412c1-521e-4c45-aa0d-899eec4ff4bc',
  'has_ml_model': True,
  'has_onnx_model': True,
  'has_tflite_model': True,
  'has_labels_file': True,
  'shared_with_me': False,
  'auto_convert': True,
  'privacy_enabled': False}]

In [26]:
# Get the application_id for your framework (version).
application_id = cl.get_application_id(
    base_framework="pytorch",
    framework="fastai",
    base_framework_version=str(torch.__version__),
    framework_version=str(fastai.__version__),
    application="image_classification"
)

In [27]:
new_version = {
    "name": "Flags demo from SDK",
    "application_id": application_id,
    "dataset_version_id":dataset_id
}

In [29]:
flag_demo_version = cl.create_model_version(flag_model['id'], new_version)

In [30]:
flag_demo_version

{'id': 'c8fcaf1e-9214-4ca4-91f2-77e92697fa58',
 'created_at': '2021-10-14T18:02:21.680347045Z',
 'update_at': '2021-10-14T18:02:21.680347045Z',
 'deleted_at': None,
 'name': 'Flags demo from SDK',
 'description': '',
 'model_id': '94f0fb9b-c93d-4078-863a-415ddc154cb6',
 'classes': '',
 'count': 0,
 'precision': 0,
 'recall': 0,
 'accuracy': 0,
 'user_id': 'd7159432-f218-44ac-aebe-e5d661d62862',
 'can_inference': False,
 'has_logo': False,
 'logo': '',
 'public': False,
 'config': '',
 'application_id': '9933abf0-f31d-4a06-9433-aab0d2ca5d26',
 'version': '',
 'version_number': 9,
 'has_ml_model': False,
 'has_onnx_model': False,
 'has_tflite_model': False,
 'has_labels_file': False,
 'dataset_version_id': 'f3d3043a-9344-4365-8147-8b6f3818a95f',
 'training_request_id': ''}

In [31]:
cl.upload_model_version(flag_demo_version)

{'id': 'c8fcaf1e-9214-4ca4-91f2-77e92697fa58',
 'created_at': '2021-10-14T18:02:21.680347Z',
 'update_at': '2021-10-14T18:03:39.152003936Z',
 'deleted_at': None,
 'name': 'Flags demo from SDK',
 'description': '',
 'model_id': '94f0fb9b-c93d-4078-863a-415ddc154cb6',
 'classes': '',
 'count': 0,
 'precision': 0,
 'recall': 0,
 'accuracy': 0,
 'user_id': 'd7159432-f218-44ac-aebe-e5d661d62862',
 'can_inference': True,
 'has_logo': False,
 'logo': '',
 'public': False,
 'config': '',
 'application_id': '9933abf0-f31d-4a06-9433-aab0d2ca5d26',
 'version': '',
 'version_number': 9,
 'has_ml_model': True,
 'has_onnx_model': True,
 'has_tflite_model': True,
 'has_labels_file': True,
 'dataset_version_id': 'f3d3043a-9344-4365-8147-8b6f3818a95f',
 'training_request_id': ''}

In [24]:
ls

58f29d19-b0bc-43bc-af47-f5a82c677f59.names
58f29d19-b0bc-43bc-af47-f5a82c677f59.onnx
README.md
bus-car-person.jpg
data/
data2/
dogs_download/
flag1.jpeg
flag2.png
flag3.jpg
images/
land_SAT_download/
object-detection-edge.ipynb
seeme-quick-guide-fastai-v1.ipynb
seeme-quick-guide-fastai-v2-data.ipynb
seeme-quick-guide-fastai-v2.ipynb
seeme-quick-guide-fastai-with-dataset-version-Copy1.ipynb
seeme-quick-guide-fastai-with-dataset-version.ipynb
seeme_quick_guide_fastai_with_dataset_version_v2.ipynb
un_flags/
v4_dogs/


In [32]:
image_location = "flag1.jpeg"

In [35]:
# Here, we will use the Python SDK to classify our test image
result = cl.version_inference(flag_demo_version, image_location)

In [36]:
# Print out the results
print(result["prediction"])
print(result["confidence"])

France
0.9957467


Make sure to check if they match the predictions from our model in the notebook.